In [43]:
import mysql.connector

dbconfig = {
        'host': 'ich-db.edu.itcareerhub.de',
        'user': 'ich1',
        'password': 'password',
        'database': 'ich_edit'
    }

df_info = {"users" : ["id","name","age"],
           "product" : ["pid","prod","quantity"],
           "sales" : ["sid","pid","id"]}

list_of_tables = df_info.keys()

def query(func):
    def wrapper(*args):
        connection = mysql.connector.connect(**dbconfig)
        cursor = connection.cursor()
        
        _sql = func(*args)
        
        cursor.execute(_sql)
        result = cursor.fetchall()
        
        if len(result) > 0:
            for row in result:
                print(*row, sep="\t\t")
        else:
            print("Ничего не найдено")
                
        cursor.close()
        connection.close()
    return wrapper
    

def check_input(tables):
    if not set(tables).issubset(list_of_tables):
        print("Введены некорректные имена таблицы")
        return False
    if set(tables) == {"users","products"}:
        print('Эти таблицы не связаны')
        return False
    return set(tables)

@query
def input_fields(table):

    fields = input(f"Выберите одно или несколько полей {df_info[table]} этой таблицы или введите * для выбора всех: ").strip().split()
    if fields != ["*"] and not set(fields).issubset(df_info[table]):
        print("Введено некорректное значение. Будет использовано значение *")
    elif fields != ["*"] and len(set(fields)) == 1:
        value = input(f"Введите искомое значение поля {fields[0]} таблицы {table} или '0', если нужно вывести все значения: ")
    if value != "0":
            sign = input(f"Введите знак - больше, меньше или равно: ").strip()
            if sign not in ["<",">",">=","<=","="]:
                print("Введено некорректное значение. Будет использовано значение =")
                sign = "="
            return f"SELECT * FROM {table} WHERE {fields[0]}{sign}'{value}'"
    elif len(set(fields)) == 2:
        fields = list(set(fields))
        return f"SELECT {fields[0]}, {fields[1]} FROM {table}"
    
    return f"SELECT * FROM {table}"

        
@query
def join_tables(tables):
    if tables == {"users", "sales"}:
        return "SELECT * FROM users JOIN sales ON users.id = sales.id"
    elif tables == {'sales','product'}:
        return "SELECT * FROM sales JOIN product ON sales.pid = product.pid"
    elif tables == set(list_of_tables):
        return "SELECT * FROM users JOIN sales ON users.id=sales.id JOIN product ON product.pid = sales.pid"
    
    
if __name__ == "__main__":
    str_of_tables = ", ".join(list_of_tables)
    
    tables = input(f"Введите таблицу или список таблиц из набора {str_of_tables}: ").split()
    if tables := check_input(tables):
        if len(tables) == 1:
            input_fields(list(tables)[0])
        else:
            join_tables(tables)
    
        

Введите таблицу или список таблиц из набора users, product, sales: sales
Выберите одно или несколько полей ['sid', 'pid', 'id'] этой таблицы или введите * для выбора всех: id
Введите искомое значение поля id таблицы sales или '0', если нужно вывести все значения: 3
Введите знак - больше, меньше или равно: >
1		1		6
5		2		6
4		1		7
2		2		10
3		3		10
6		2		83
7		3		99
